<a href="https://colab.research.google.com/github/diogopaz/projeto-marvel/blob/TesteCommit/projeto_marvel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando bibliotecas necessárias

In [3]:
#!pip install dotenv

In [4]:
import requests
import hashlib
import time
import os
import pandas as pd
from dotenv import load_dotenv
import sqlite3

# Carregando variáveis de ambiente
Estamos fazendo upload do arquivo .env com as chaves de acesso à API no ambeinte de execução do colab.

In [5]:
load_dotenv(dotenv_path='/content/.env')
public_key = os.getenv('MARVEL_PUBLIC_KEY')
private_key = os.getenv('MARVEL_PRIVATE_KEY')

# Montando os parâmetros para realizar a requisição

In [37]:
ts = str(time.time())
to_hash = ts + str(private_key) + str(public_key)
hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

params = {
    'apikey': public_key,
    'ts': ts,
    'hash': hash_md5,
    'limit': 5
}

# Realizando a requisição

In [42]:
url = 'https://gateway.marvel.com/v1/public/characters/1009610'
response = requests.get(url, params=params)
data = response.json()

In [43]:
data

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2025 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2025 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2025 MARVEL</a>',
 'etag': 'b1ab6fe1735aacdf908357174dc82ec30f9ef9f7',
 'data': {'offset': 0,
  'limit': 5,
  'total': 1,
  'count': 1,
  'results': [{'id': 1009610,
    'name': 'Spider-Man (Peter Parker)',
    'description': 'Bitten by a radioactive spider, high school student Peter Parker gained the speed, strength and powers of a spider. Adopting the name Spider-Man, Peter hoped to start a career using his new abilities. Taught that with great power comes great responsibility, Spidey has vowed to use his powers to help people.',
    'modified': '2021-06-30T17:32:26+0000',
    'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/3/50/526548a343e4b',
     'extension': 'jpg'},
    'resourceURI': 'https://gateway.marvel.com/v1/public/characters/1009610',
    'comics': {'available':

In [11]:
df_ini = pd.DataFrame(data['data']['results'])
df_ini.sample(20)

,id,name,description,modified,thumbnail,resourceURI,comics,series,stories,events,urls
78,1010336,Atlas (Team),,1970-01-01T00:00:00+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 8, 'collectionURI': 'https://gat...","{'available': 5, 'collectionURI': 'https://gat...","{'available': 6, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
36,1009567,Amanda Sefton,,2014-06-26T13:02:19+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 7, 'collectionURI': 'https://gat...","{'available': 2, 'collectionURI': 'https://gat...","{'available': 7, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
93,1011354,Barracuda,,1970-01-01T00:00:00+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 4, 'collectionURI': 'https://gat...","{'available': 3, 'collectionURI': 'https://gat...","{'available': 5, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
56,1010801,Ant-Man (Scott Lang),,2017-01-31T11:03:40+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 137, 'collectionURI': 'https://g...","{'available': 58, 'collectionURI': 'https://ga...","{'available': 163, 'collectionURI': 'https://g...","{'available': 2, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
44,1016824,Ancient One (Ultimate),,2012-07-10T19:15:49+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 2, 'collectionURI': 'https://gat...","{'available': 2, 'collectionURI': 'https://gat...","{'available': 1, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
73,1010748,Arsenic,,1970-01-01T00:00:00+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 0, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
82,1009165,Avengers,Earth's Mightiest Heroes joined forces to take...,2020-07-21T10:29:09+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 2298, 'collectionURI': 'https://...","{'available': 705, 'collectionURI': 'https://g...","{'available': 3129, 'collectionURI': 'https://...","{'available': 32, 'collectionURI': 'https://ga...","[{'type': 'detail', 'url': 'http://marvel.com/..."
35,1011227,Amadeus Cho,,2013-08-07T13:50:56+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 150, 'collectionURI': 'https://g...","{'available': 33, 'collectionURI': 'https://ga...","{'available': 179, 'collectionURI': 'https://g...","{'available': 5, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
51,1009346,Anne Marie Hoag,,1970-01-01T00:00:00+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available': 0, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","{'available': 0, 'collectionURI': 'https://gat...","[{'type': 'detail', 'url': 'http://marvel.com/..."
31,1009435,Alicia Masters,,2010-11-04T15:22:16+0000,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,https://gateway.marvel.com/v1/public/character...,"{'available':